In [1]:
from mxnet import gluon, nd
from mxnet.gluon import nn


class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    
    #将输入减掉均值后输出的层
    def forward(self, x):
        return x - x.mean()
    
#实例化该层
layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))

D:\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [3]:
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

In [6]:
net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

D:\anaconda3\lib\site-packages\mxnet\gluon\parameter.py:862: UserWarning: Parameter 'dense1_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
D:\anaconda3\lib\site-packages\mxnet\gluon\parameter.py:862: UserWarning: Parameter 'dense1_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


5.6024874e-10

In [4]:
# 含模型参数的自定义层
params = gluon.ParameterDict()
# 可以通过get函数从ParameterDict创建Parameter实例。
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [7]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [6]:
# 输入个数5，输出个数3
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [7]:
# 直接使用自定义层做前向计算。
dense.initialize()
# 随机两个参数，每个参数由5个组成
dense(nd.random.uniform(shape=(2, 5)))


[[0.06917784 0.01627153 0.01029644]
 [0.02602214 0.04537309 0.        ]]
<NDArray 2x3 @cpu(0)>

In [8]:
# 使用自定义层构造模型
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.03820475]
 [0.04035058]]
<NDArray 2x1 @cpu(0)>

In [9]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)
        
dense = MyDense(units=3, in_units=5)
dense.initialize()
dense(nd.random.uniform(shape=(1000, 5)))


[[9.3780458e-05 0.0000000e+00 7.4412957e-02]
 [1.1123408e-02 5.5128459e-02 1.3976939e-02]
 [1.8191710e-04 3.8248748e-03 1.5906963e-02]
 ...
 [0.0000000e+00 5.5083368e-02 3.4891628e-02]
 [0.0000000e+00 4.6397895e-03 7.5148061e-02]
 [2.7498063e-03 0.0000000e+00 8.7269217e-02]]
<NDArray 1000x3 @cpu(0)>